In [2]:
import pickle
import gzip
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import time

### Analysis of the Images (one Tumor per Image)

#### Creation of Training, Validation and Testset
Approx 60% is for training, 20% for validation and 20% for testing. The borders are choosen so that no patient is splitt between the different sets.

In [70]:
start = time.time()
#Inselspital
with gzip.open('GBM_tumors.pickle.gz') as f:
    Names,X,Y = pickle.load(f)
print ("Loaded data in " + str(time.time() - start))
print ("   " + str(X.shape) + " y " + str(Y.shape))


gmb_valid = 306
gbm_test = 408

Y_train = Y[0:gmb_valid]
Y_valid = Y[gmb_valid:gbm_test]
Y_test  = Y[gbm_test:]

X_train = X[0:gmb_valid,:,:,:]
X_valid = X[gmb_valid:gbm_test,:,:,:]
X_test  = X[gbm_test:,:,:]

Loaded data in 0.093239068985
   (500, 1, 48, 48) y (500,)


In [71]:
start = time.time()
#Inselspital
with gzip.open('META_tumors.pickle.gz') as f:
    Names,X,Y = pickle.load(f)
print ("Loaded data in " + str(time.time() - start))
print ("   " + str(X.shape) + " y " + str(Y.shape))


meta_valid = 247
meta_test = 340

Y_train = np.append(Y_train, Y[0:meta_valid])
Y_valid = np.append(Y_valid, Y[meta_valid:meta_test])
Y_test = np.append(Y_test, Y[meta_test:])

X_train = np.concatenate((X_train, X[0:meta_valid,:,:,:]), axis=0)
X_valid = np.concatenate((X_valid, X[meta_valid:meta_test,:,:,:]), axis=0)
X_test = np.concatenate((X_test, X[meta_test:,:,:,:]), axis=0)

Loaded data in 0.0828847885132
   (429, 1, 48, 48) y (429,)


In [92]:
# Mixed it up it seems that only one set is needed for training, 
# split is done automatically in cafe

X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((Y_train, Y_test), axis=0)

#### Random Permutations of the training set
In order that the minibatch sees as much variation of the data as possible, we permute the trainingset.

In [93]:
perm = np.random.permutation(len(y))
X = X[perm,:,:,:]
y = y[perm]
y = y - 2
y = np.asarray(y,dtype='int32')
y

array([1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0,

In [94]:
print(str(np.shape(X)) + " " + str(np.shape(y)))
print(str(np.shape(X_test)) + " " + str(np.shape(Y_test)))

(734, 1, 48, 48) (734,)
(181, 1, 48, 48) (181,)


#### The first try

In [95]:
PIXELS = 48

from lasagne import layers
from lasagne import nonlinearities
from nolearn.lasagne import NeuralNet

net1 = NeuralNet(
    # Geometry of the network
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('hidden4', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, PIXELS, PIXELS), #None in the first axis indicates that the batch size can be set later
    conv1_num_filters=32, conv1_filter_size=(3, 3), pool1_pool_size=(2, 2), #pool_size used to be called ds in old versions of lasagne
    conv2_num_filters=64, conv2_filter_size=(2, 2), pool2_pool_size=(2, 2),
    hidden4_num_units=500,
    output_num_units=2, output_nonlinearity=nonlinearities.softmax,

    # learning rate parameters
    update_learning_rate=0.01,
    update_momentum=0.9,
    regression=False,
    # We only train for 10 epochs
    max_epochs=10,
    verbose=1,

    # Training test-set split
    eval_size = 0.2
    )

In [96]:
net = net1.fit(X,y)

/Library/Python/2.7/site-packages/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "
/Library/Python/2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


  DenseLayer        	(None, 2)           	produces       2 outputs
  DenseLayer        	(None, 500)         	produces     500 outputs
  MaxPool2DLayer    	(None, 64, 11, 11)  	produces    7744 outputs
  Conv2DLayer       	(None, 64, 22, 22)  	produces   30976 outputs
  MaxPool2DLayer    	(None, 32, 23, 23)  	produces   16928 outputs
  Conv2DLayer       	(None, 32, 46, 46)  	produces   67712 outputs
  InputLayer        	(None, 1, 48, 48)   	produces    2304 outputs


/Library/Python/2.7/site-packages/lasagne/layers/base.py:99: UserWarning: layer.get_output_shape() is deprecated and will be removed for the first release of Lasagne. Please use layer.output_shape instead.
  warnings.warn("layer.get_output_shape() is deprecated and will be "
/Library/Python/2.7/site-packages/lasagne/layers/base.py:109: UserWarning: layer.get_output(...) is deprecated and will be removed for the first release of Lasagne. Please use lasagne.layers.get_output(layer, ...) instead.
  warnings.warn("layer.get_output(...) is deprecated and will be "


KeyboardInterrupt: 